# STATIC GRAPH

In [1]:
import tensorflow as tf
from tensorflow.contrib.layers import xavier_initializer as xinit

## DATA

In [2]:
import data_utils
metadata, idx_q, idx_a = data_utils.load_data('../data/')

In [3]:
# add special symbol
i2w = metadata['idx2w'] + ['GO']
w2i = metadata['w2idx']
w2i['GO'] = len(i2w)-1

## Parameters

In [4]:
B = 500
L = len(idx_q[0])
vocab_size = len(i2w)
enc_hdim = 250
dec_hdim = enc_hdim

## Graph

In [5]:
tf.reset_default_graph()

### Placeholders

In [6]:
inputs = tf.placeholder(tf.int32, shape=[B,L], name='inputs')
targets = tf.placeholder(tf.int32, shape=[B,L], name='targets')
decoder_inputs = tf.concat(
    values=[tf.constant(w2i['GO'], dtype=tf.int32, shape=[B,1]), targets[:, 1:]],
    axis=1)
training = tf.placeholder(tf.bool, name='is_training')

In [7]:
decoder_inputs, targets, inputs

(<tf.Tensor 'concat:0' shape=(500, 20) dtype=int32>,
 <tf.Tensor 'targets:0' shape=(500, 20) dtype=int32>,
 <tf.Tensor 'inputs:0' shape=(500, 20) dtype=int32>)

## Embedding

In [8]:
emb_mat = tf.get_variable('emb', shape=[vocab_size, enc_hdim], dtype=tf.float32, 
                         initializer=xinit())
emb_enc_inputs = tf.nn.embedding_lookup(emb_mat, inputs)
emb_dec_inputs = tf.nn.embedding_lookup(emb_mat, decoder_inputs)

## Encoder

In [9]:
with tf.variable_scope('encoder'):
    encoder_cell = tf.contrib.rnn.LSTMCell(num_units=enc_hdim)
    encoder_outputs, context = tf.nn.dynamic_rnn(cell=encoder_cell,
                          inputs=emb_enc_inputs, 
                          initial_state =
                                encoder_cell.zero_state(batch_size=B, dtype=tf.float32)
                         )

## Decoder

In [10]:
# inference - custom rnn
def inference_decoder(cell, state):
    #tf.get_variable_scope().reuse_variables()
    with tf.variable_scope('decoder_1'):
        dec_outputs = []
        input_ = tf.constant(w2i['GO'], dtype=tf.int32, shape=[B,])
        input_ = tf.nn.embedding_lookup(emb_mat, input_)
        
        for i in range(L):
            decoder_output, state = cell(input_, state)
            dec_outputs.append(decoder_output)
            input_ = decoder_output

    return tf.stack(dec_outputs), None

In [11]:
with tf.variable_scope('decoder') as scope:
    decoder_cell = tf.contrib.rnn.LSTMCell(num_units=dec_hdim)
    
    decoder_outputs, _ = tf.nn.dynamic_rnn(cell=decoder_cell,
                          inputs=emb_dec_inputs, 
                          initial_state= context,
                          scope='decoder_1')
                         
    tf.get_variable_scope().reuse_variables()
    
    decoder_outputs_inf, _ = inference_decoder(decoder_cell, context)
    #decoder_outputs_inf, _ = inference_decoder(decoder_cell, context)
    '''
    decoder_outputs_inf, _ = tf.nn.dynamic_rnn(cell=decoder_cell,
                          inputs=emb_dec_inputs, 
                          initial_state= context
                         )
    '''

## Logits and Probabilities

In [12]:
Wo = tf.get_variable('Wo', shape=[dec_hdim, vocab_size], dtype=tf.float32, 
                         initializer=xinit())
bo = tf.get_variable('bo', shape=[vocab_size], dtype=tf.float32, 
                         initializer=xinit())
proj_outputs = tf.matmul(tf.reshape(decoder_outputs, [B*L, dec_hdim]), Wo) + bo
proj_outputs_inf = tf.matmul(tf.reshape(decoder_outputs_inf, [B*L, dec_hdim]), Wo) + bo
logits = tf.reshape(proj_outputs_inf, [B, L, vocab_size])
probs = tf.nn.softmax(logits)

## Loss

In [13]:
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(
    logits =  tf.reshape(proj_outputs, [B, L, vocab_size]),
    labels = targets)
loss = tf.reduce_mean(cross_entropy)

## Optimizatin

In [14]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
train_op = optimizer.minimize(loss)

## Inference

In [15]:
prediction = tf.argmax(probs, axis=-1)

# TRAINING

In [16]:
config = tf.ConfigProto(allow_soft_placement = True)
sess = tf.InteractiveSession(config = config)
sess.run(tf.global_variables_initializer())

### Training parameters

In [17]:
num_epochs = 20

## Begin

In [18]:
for i in range(num_epochs):
    avg_loss = 0.
    for j in range(len(idx_q)//B):
        _, loss_v = sess.run([train_op, loss], feed_dict = {
            inputs : idx_q[j*B:(j+1)*B],
            targets : idx_a[j*B:(j+1)*B]
        })
        avg_loss += loss_v
        if j and j%30==0:
            print('{}.{} : {}'.format(i,j,avg_loss/30))
            avg_loss = 0.

0.30 : 3.9470930178960164
0.60 : 2.546596145629883
0.90 : 1.7961014747619628
0.120 : 1.1721597969532014
0.150 : 0.8484559178352356
0.180 : 0.6722201824188232
0.210 : 0.5757172624270122
0.240 : 0.49877248803774515
1.30 : 0.4234134624401728
1.60 : 0.3767172416051229
1.90 : 0.35226030747095743
1.120 : 0.3281350791454315
1.150 : 0.3169568806886673
1.180 : 0.3092158039410909
1.210 : 0.3064297099908193
1.240 : 0.3026467909415563
2.30 : 0.30772637327512103
2.60 : 0.2944767475128174
2.90 : 0.29480880002180737
2.120 : 0.2905799627304077
2.150 : 0.289568629860878
2.180 : 0.2882703880469004
2.210 : 0.29311469197273254
2.240 : 0.2900694638490677
3.30 : 0.2999609112739563
3.60 : 0.2885451485713323
3.90 : 0.2894310474395752
3.120 : 0.28706316550572714
3.150 : 0.286658838391304
3.180 : 0.285483052333196
3.210 : 0.28860851128896076
3.240 : 0.28708851933479307
4.30 : 0.29843737185001373
4.60 : 0.2876114845275879
4.90 : 0.2882773111263911
4.120 : 0.28662173052628837
4.150 : 0.285749947031339
4.180 : 0.2

In [19]:
j = 7 
pred_v = sess.run(prediction, feed_dict = {
            inputs : idx_q[j*B:(j+1)*B],
            targets : idx_a[j*B:(j+1)*B]
        })

In [ ]:
pred_v[0], idx_a[j*B:(j+1)*B][0]

In [20]:
def arr2sent(arr):
    return ' '.join([i2w[item] for item in arr])

In [24]:
for i in range(B):
    print(arr2sent(pred_v[i]))

i i i i i i i i i i i i i i i i thank i i i
i i i i i i i i i i i i i i i i thanks i i thank
i i i i i i i i thanks i i i i i i i i i i i
i i i i i thank i i i i i i i i science i i i i i
i i i i i i i i i thank i i i i i i i i i i
i i i i i i i i i i i thank i i i thank i i i thanks
i thank i i i i thank i i i i i i i thank i i i i i
i i i i i i i thank i i i i i i i i i i i i
i i i i i this i i i i i i i i i i i i i i
i i thank i i i i thank i i i i i i i i i i i i
i i i i i i i i i i i i i i i i i i i i
i i i i i i i i i i i i i i i i i i i i
i i i i i i i i i i i i i i i i i i i i
i i i i i i i i i i i i i i i i thank i i i
i i i i i i i i i i i i i i i i i i i this
i i i i i i i i i i i i i i i i i i i i
i i i i i thank i i i i i thanks i i i thank i i i i
i i i i i i i i i i i i i i thank i i i i i
i i i i i i i i i i i i i i i thank i i i i
i i i i thank i i i i i i i i i i i i i i i
i i thanks i i thank i i i i i i i i i i i i i i
i i i i i i i i i i i thank i i i i i i i i
tha

In [22]:
arr2sent( idx_a[j*B:(j+1)*B][11])

'hey whats that in the background racist _ _ _ _ _ _ _ _ _ _ _ _ _'